In [ ]:
#@title install
!git clone https://github.com/lingxusb/megaDNA.git
%cd megaDNA

!pip install .
!wget https://huggingface.co/lingxusb/megaDNA_updated/resolve/main/megaDNA_phage_145M.pt

Cloning into 'megaDNA'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 126 (delta 35), reused 24 (delta 5), pack-reused 42 (from 1)
Receiving objects: 100% (126/126), 520.62 KiB | 3.64 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/megaDNA
Processing /content/megaDNA
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
#@title generate sequences
import torch

seqence_length = 500 #@param {type:"integer"}
seqence_number = 1 #@param {type:"integer"}

import torch
import numpy as np

nucleotides = ['**', 'A', 'T', 'C', 'G', '#'] # vocabulary
def token2nucleotide(s):
    return nucleotides[s]

PRIME_LENGTH = 4 # give the model a random DNA primer to start
model_path = "megaDNA_phage_145M.pt" # model name
device = 'cuda'

for j in range(seqence_number):
    # Load the pre-trained model
    model = torch.load(model_path, map_location=torch.device(device), weights_only=False)

    model.eval()  # Set the model to evaluation mode

    # set the random DNA primer
    primer_sequence = torch.tensor(np.random.choice(np.arange(1,5), PRIME_LENGTH)).long().to(device)[None,]
    primer_DNA = ''.join(map(token2nucleotide, primer_sequence[0]))
    print(f"Primer sequence: {primer_DNA}\n{'*' * 100}")

    # Generate a sequence using the model
    seq_tokenized = model.generate(primer_sequence,
                                   seq_len=seqence_length,
                                   temperature=0.95,
                                   filter_thres=0.0)
    generated_sequence = ''.join(map(token2nucleotide, seq_tokenized.squeeze().cpu().int()))
    print(f"Generated sequence: {generated_sequence}")

    # Split the generated sequence into contigs at the '#' character
    contigs = generated_sequence.split('#')

    # Write the contigs to a .fna file
    output_file_path = f"generate_{1+j}.fna"
    with open(output_file_path, "w") as file:
        for idx, contig in enumerate(contigs):
            if len(contig) > 0:
                file.write(f">contig_{idx}\n{contig}\n")

    # Clean up to free memory
    del model, primer_sequence, generated_sequence
    torch.cuda.empty_cache()


Primer sequence: TCGT
****************************************************************************************************


  0%|          | 0/496 [00:00<?, ?it/s]

Generated sequence: TCGTTGATTTCTTCCTGGTATTCGCGCTGCGCGATCGTGTCGAACGAGGCTTTCAGTTCTTCGTCGAATTTCGACGCTTCTTCGGCGGCGATCAGTTTGTCGGCAACCTTCGCGTTCTTCTGTTCCTTGATACCCTCTTTGCGTGTCTTCACGTCCTCAATGTTCAGCCGCATTTGATCGTCCAGCTTGTTTTGCAGGGTGCGGATCAAGCGAATTTTTGTTTCCATTTTGTGCCCTCCTTATATGTGAAAAGATTGATTTCCGATTTTTCCTGCGGCGCACCGGTCGTTTCCTGCGTTATGCGGACACCTCGCAAGGCTTTTGTAAATAGTTAATGTGTAAGCGCACCCGCAAGTAGGCGTACACCTCTTCGTAGACTGTCATAAATTCGCCGTAGCCGAACGCCTTTGCAAAGTCGATCCCCGCCGCCGAAAGATCGAGTGCGTAGGCGACGGCGTTTTCGTTTTCTCCAACGACGTATTCCACCGTTACCCCTGCCG


In [ ]:
#@title download generated sequences
import os
from google.colab import files

# Name for the zip file
zip_name = "generated_seq.zip"

# Command to zip all .fna files in the current directory
os.system(f"zip {zip_name} *.fna")

# Download the zip file
files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#simulated output

In [ ]:
import torch
import numpy as np

# Define the vocabulary for nucleotides with special tokens
nucleotides = ['**', 'A', 'T', 'C', 'G', '#']

def token2nucleotide(token):

    return nucleotides[token]

def generate_sequence(model_path, sequence_length, primer_length, device='cuda'):

    # Load the pre-trained model from the specified path
    model = torch.load(model_path, map_location=torch.device(device), weights_only=False)
    model.eval()  # Ensure the model is in evaluation mode


    primer = torch.tensor(np.random.choice(np.arange(1, 5), primer_length)).long().to(device)[None,]
    primer_str = ''.join(token2nucleotide(token) for token in primer.squeeze().cpu().numpy())
    print(f"Primer sequence: {primer_str}\n{'*' * 100}")


    seq_tokenized = model.generate(primer,
                                   seq_len=sequence_length,
                                   temperature=0.95,
                                   filter_thres=0.0)
    generated_sequence = ''.join(token2nucleotide(token) for token in seq_tokenized.squeeze().cpu().int())


    contigs = generated_sequence.split('#')

    # Write the contigs to a FASTA (.fna) file
    output_file = "generated_sequence.fna"
    with open(output_file, "w") as file:
        for idx, contig in enumerate(contigs):
            if contig:  # Only write non-empty contigs
                file.write(f">contig_{idx}\n{contig}\n")

    # Free up GPU memory
    del model
    torch.cuda.empty_cache()

    return primer_str, generated_sequence, contigs, output_file

# -----------------------------
# Parameters for generation
sequence_length = 500
primer_length = 4
model_path = "megaDNA_phage_145M.pt"

# Generate the sequence
primer_str, generated_sequence, contigs, output_file = generate_sequence(model_path, sequence_length, primer_length)

# -----------------------------
# Simulated Output Display
print("\n--- Generated Sequence ---")
print(generated_sequence)

print("\n--- Contigs (Preview) ---")
for idx, contig in enumerate(contigs):

    preview = contig[:50] + "..." if len(contig) > 50 else contig
    print(f"Contig {idx}: {preview}")

print(f"\nFASTA file saved to: {output_file}")


Primer sequence: GGAA
****************************************************************************************************


  0%|          | 0/496 [00:00<?, ?it/s]

/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)



--- Generated Sequence ---
GGAAGTTAGATACTTAAGTAAAGACTGCAAACCCTTTTTACAGCTGTTGGGTATCGGCAGTTGTCTTACTACTAACGTAGACAATTAGCAAGTTATAGTGAAAAGCTTTATACTTCCCATAATTACCATTACAGGTTATTGCCGCTCTTCTAAAGTCATATAATGCCTGCCGCAGGTCGCAAGCATCACTATTTGCCTGATAAAATCCTCTGTGTATAAAGATGGCTTACAAATAGAAGCAGGGACATTACCGGCGGAAAAAGCAATTAATTACTTTAAAAGCCAAAGTTGCATTATGCAACGCGTAAACAGGTCACGGCAAAAAGAACGCTTTACGTCTTTACCTACTACGGTGGATAGATAAGCGGGTATATATTCCTATGATCCGGGAGGTTTCCGGGGATGAGCCGCAAAGTAACCGAAAAATGCATCAACATCCTGGGCTACGGGAAAATGACGGTATCCCTTACGCTGGAGGAAGTAGCCGAGTATGCGGCCGG

--- Contigs (Preview) ---
Contig 0: GGAAGTTAGATACTTAAGTAAAGACTGCAAACCCTTTTTACAGCTGTTGG...

FASTA file saved to: generated_sequence.fna


#simulated gap fillings

In [ ]:
import torch
import numpy as np

# Define the vocabulary and mapping functions
nucleotides = ['**', 'A', 'T', 'C', 'G', '#']

def token2nucleotide(token):

    return nucleotides[token]

def nucleotide2token(nuc):

    mapping = {'A': 1, 'T': 2, 'C': 3, 'G': 4}
    return mapping.get(nuc.upper(), None)

def fill_gap(input_sequence, model_path, device='cuda', temperature=0.95):

    gap_start = input_sequence.find('N')
    gap_end = input_sequence.rfind('N') + 1
    if gap_start == -1:
        raise ValueError("Input sequence does not contain a gap ('N').")

    left_context = input_sequence[:gap_start]
    right_context = input_sequence[gap_end:]
    gap_length = gap_end - gap_start

    print(f"Left context: {left_context}")
    print(f"Gap length: {gap_length}")
    print(f"Right context: {right_context}")
    print(f"{'*' * 100}")

    left_tokens = []
    for nuc in left_context:
        token = nucleotide2token(nuc)
        if token is None:
            raise ValueError(f"Invalid nucleotide '{nuc}' in left context.")
        left_tokens.append(token)
    # Convert to tensor and add batch dimension
    primer_sequence = torch.tensor(left_tokens).long().to(device)[None,]

    # Load the pre-trained model
    model = torch.load(model_path, map_location=torch.device(device), weights_only=False)
    model.eval()  # Set the model to evaluation mode


    gap_tokens = model.generate(primer_sequence,
                                seq_len=gap_length,
                                temperature=temperature,
                                filter_thres=0.0)

    # Convert generated tokens to a gap sequence
    gap_sequence = ''.join(token2nucleotide(token) for token in gap_tokens.squeeze().cpu().int())

    # Construct the final sequence by filling the gap
    final_sequence = left_context + gap_sequence + right_context

    # save the final sequence to a FASTA file
    output_file = "gap_filled_sequence.fna"
    with open(output_file, "w") as file:
        file.write(">filled_gap_sequence\n")
        file.write(final_sequence + "\n")

    # Free GPU memory and clean up
    del model, primer_sequence, gap_tokens
    torch.cuda.empty_cache()

    return final_sequence, gap_sequence, output_file

# -----------------------------
input_seq = "ATCGATCGNNNNATGCGTACG"
model_path = "megaDNA_phage_145M.pt"  # pre-trained model path

# Fill in the gap
final_seq, generated_gap, fasta_file = fill_gap(input_seq, model_path, device='cuda', temperature=0.95)

# Display results
print("\n--- Generated Gap Sequence ---")
print(generated_gap)

print("\n--- Final Sequence (Gap Filled) ---")
print(final_seq)

print(f"\nFASTA file saved to: {fasta_file}")


Left context: ATCGATCG
Gap length: 4
Right context: ATGCGTACG
****************************************************************************************************


0it [00:00, ?it/s]


--- Generated Gap Sequence ---
ATCGATCG

--- Final Sequence (Gap Filled) ---
ATCGATCGATCGATCGATGCGTACG

FASTA file saved to: gap_filled_sequence.fna
